<a href="https://colab.research.google.com/github/SherifNabil-Eng/AzureCognitivePoCs/blob/main/TestFormRecLayout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install azure-ai-formrecognizer==3.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.4/228.4 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 1.4 MB/s eta 0:00:00


In [ ]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 1.2 MB/s eta 0:00:00


In [ ]:
# import libraries
from google.colab import drive
drive.mount('/content/gdrive')

import os
import numpy as np
import pandas as pd
from datetime import date
import sys
#reload(sys)


import os
import locale
os.environ["PYTHONIOENCODING"] = "utf-8"


from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError


os.chdir("/content/gdrive/My Drive/Colab Notebooks/Tests")
#sys.setdefaultencoding('utf-8')
!locale-gen en_US.UTF-8
!export LANG=en_US.UTF-8 LANGUAGE=en_US.en
!LC_ALL=en_US.UTF-8

Mounted at /content/gdrive
Generating locales (this might take a while)...
  en_US.UTF-8... done
Generation complete.


In [ ]:
# coding: utf-8

# -------------------------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License. See License.txt in the project root for
# license information.
# --------------------------------------------------------------------------

"""
FILE: sample_analyze_layout.py
DESCRIPTION:
    This sample demonstrates how to extract text, selection marks, and layout information from a document
    given through a file.
    Note that selection marks returned from begin_analyze_document(model_id="prebuilt-layout") do not return the text
    associated with the checkbox. For the API to return this information, build a custom model to analyze the
    checkbox and its text. See sample_build_model.py for more information.
USAGE:
    python sample_analyze_layout.py
    Set the environment variables with your own values before running the sample:
    1) AZURE_FORM_RECOGNIZER_ENDPOINT - the endpoint to your Form Recognizer resource.
    2) AZURE_FORM_RECOGNIZER_KEY - your Form Recognizer API key
"""




def format_polygon(polygon):
    if not polygon:
        return "N/A"
    return ", ".join(["[{}, {}]".format(p.x, p.y) for p in polygon])


def analyze_layout(fund_file):
    #path_to_sample_documents = os.path.abspath(
    #    os.path.join(
    #        os.path.abspath(__file__),
    #        "..",
     #       "..",
     #       "./sample_forms/forms/form_selection_mark.png",
     #   )
    #)

    currroot=os.getcwd() 
    #print("File location using os.getcwd():", os.getcwd())
    path_to_sample_documents = os.path.join(currroot, fund_file)
    #print ("path to form is ",path_to_sample_documents)
     
    

    #endpoint = os.environ["AZURE_FORM_RECOGNIZER_ENDPOINT"]
    #key = os.environ["AZURE_FORM_RECOGNIZER_KEY"]

    endpoint = "https://cognservgen.cognitiveservices.azure.com/"
    key = "fba635423d6547248dd57386dd6220e4"

    document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key)
    )
    with open(path_to_sample_documents, "rb") as f:
        poller = document_analysis_client.begin_analyze_document(
            "prebuilt-layout", document=f
        )
    result = poller.result()

    for idx, style in enumerate(result.styles):
        print(
            "Document contains {} content".format(
                "handwritten" if style.is_handwritten else "no handwritten"
            )
        )

    for page in result.pages:
        print("----Analyzing layout from page #{}----".format(page.page_number))
        print(
            "Page has width: {} and height: {}, measured with unit: {}".format(
                page.width, page.height, page.unit
            )
        )

        for line_idx, line in enumerate(page.lines):
            words = line.get_words()
            #print(
            #    "...Line # {} has word count {} and text '{}' within bounding polygon '{}'".format(
                    #line_idx,
            #        len(words),
            #        line.content,
            #        format_polygon(line.polygon),
            #    )
            #)

            #for word in words:
                #print(
                    #"......Word '{}' has a confidence of {}".format(
                        #word.content, word.confidence
                    #)
                #)

        #for selection_mark in page.selection_marks:
            #print(
                #"...Selection mark is '{}' within bounding polygon '{}' and has a confidence of {}".format(
                    #selection_mark.state,
                    #format_polygon(selection_mark.polygon),
                    #selection_mark.confidence,
                #)
            #)

    for table_idx, table in enumerate(result.tables):
        #print(
        #    "Table # {} has {} rows and {} columns".format(
        #        table_idx, table.row_count, table.column_count
        #    )
        
        #)
        fund_table =np.empty((table.row_count,table.column_count),dtype='O')
        print ("\n fund table is  ",fund_table.shape)
        #for region in table.bounding_regions:
            #print(
                #"Table # {} location on page: {} is {}".format(
                    #table_idx,
                    #region.page_number,
                    #format_polygon(region.polygon),
                #)
            #)
        for cell in table.cells:
            #print(
            #    "...Cell[{}][{}] has content '{}'".format(
            #        cell.row_index,
            #        cell.column_index,
            #        cell.content,
            #    )
            
           # )
            fund_table[cell.row_index,cell.column_index]=np.unicode_(cell.content)
            #for region in cell.bounding_regions:
            #    print(
            #        "...content on page {} is within bounding polygon '{}'".format(
            #           region.page_number,
            #            format_polygon(region.polygon),
            #        )
            #    )

    print("----------------------------------------")
    
    return fund_table
    

In [ ]:
funds = ["FormRecImgs/Stocks.png","FormRecImgs/Monetary.png","FormRecImgs/Islamic.png","FormRecImgs/Assets.png","FormRecImgs/Balanced.png"]

funds_dic = {}

if __name__ == "__main__":
    for fund_file in funds:

      try:
          fund=analyze_layout(fund_file)
          
          index_slash=fund_file.find ("/")
          index_dot=fund_file.find (".")
          fund_key = fund_file[index_slash+1:index_dot]
          
          funds_dic [fund_key] = fund 
      
      except HttpResponseError as error:
          # Examples of how to check an HttpResponseError
          # Check by error code:
          if error.error is not None:
              if error.error.code == "InvalidRequest":
                  print(f"Received an invalid request error: {error.error}")
                  sys.exit(1)
              if error.error.code == "InvalidImage":
                  print(f"Received an invalid image error: {error.error}")
                  sys.exit(1)
          # If the inner error is None and then it is possible to check the message to get more information:
          filter_msg = ["Generic error", "Timeout", "Invalid request", "InvalidImage"]
          if any(example_error.casefold() in error.message.casefold() for example_error in filter_msg):
              print(f"Uh-oh! Something unexpected happened: {error}")
              sys.exit(1)
          # Print the full error content:
          print(f"Full HttpResponseError: {error}")

Document contains handwritten content
Document contains handwritten content
Document contains handwritten content
Document contains handwritten content
----Analyzing layout from page #1----
Page has width: 3392.0 and height: 974.0, measured with unit: pixel

 fund table is   (28, 6)
----------------------------------------
Document contains handwritten content
Document contains handwritten content
Document contains handwritten content
Document contains handwritten content
Document contains handwritten content
----Analyzing layout from page #1----
Page has width: 2858.0 and height: 1073.0, measured with unit: pixel

 fund table is   (38, 6)
----------------------------------------
Document contains handwritten content
Document contains handwritten content
----Analyzing layout from page #1----
Page has width: 2850.0 and height: 561.0, measured with unit: pixel

 fund table is   (18, 6)
----------------------------------------
----Analyzing layout from page #1----
Page has width: 2826.0 a

In [ ]:
#funds_dic['Stocks']

for fund_key_temp in funds_dic:
  print (funds_dic[fund_key_temp])



[['الترتيب الاسبوعى' 'نسبة التغير الاسبوعى' 'سعر الوثيقة في 29 يناير'
  'سعر الوثيقة فى 5 فبراير' 'القيمة الاسمية' 'الصندوق']
 ['1' '-0.7728' '297.62' '295.32' '100'
  '"التجاري الدولي الثانى "استثمار"']
 ['2' '-1.2091' '934.59' '923.29' '100' 'بنك مصر الثالث']
 ['3' '-1.2093' '122.38' '120.9' '66.67' 'بنك مصر الثاني']
 ['' '-1.4491' '136.64' '134.66' '100' 'فاروس 1']
 ['5' '-2.2276' '523.44' '511.78' '' 'بنك الإسكندرية']
 ['6' '-2.4071' '265.47' '259.08' '100' 'كريدي أجريكول الثاني']
 ['7' '-2.5445' '165.85' '161.63' '10' 'بنك القاهرة الأول']
 ['8' '-2.5732' '583.33' '568.32' '100' 'كريدي أجريكول مصر الأول']
 ['9' '-2.8331' '390.03' '378.98' '100' 'المصرى الخليجي']
 ['10' '-2.8568' '31.79302' '30.88475' 'TO'
  '"البنك الأهلى المتحد " إذا""']
 ['11' '-3.1273' '21.17998' '20.51762' '' '"قناة السويس "الإجبال""']
 ['12' '-3.5184' '820.82' '791.94' '' 'MID BANK 1']
 ['13' '-3.6352' '11.13' '10.7254' '10' '"إن أي كابيتال "سهمى"']
 ['14' '-3.7492' '264.59' '254.67' '50' '"العربى الافريقى الد

In [ ]:
#get current path
currpath=os.getcwd()
print ("current path is",currpath)

current_date = today = str (date.today())

print ("current day is ",current_date)

# Create a Pandas Excel writer using XlsxWriter

output_path = 'Funds'+current_date+'.xlsx'

writer = pd.ExcelWriter(output_path, engine='xlsxwriter')

dic_keys = funds_dic.keys()

for fund_key_temp in funds_dic:
  fund_df= pd.DataFrame (funds_dic[fund_key_temp])
  fund_df["Date"] = current_date
  fund_df.to_excel(writer,sheet_name=fund_key_temp)



# Save Data to File
writer.save()
writer.close() 

current path is /content/gdrive/MyDrive/Colab Notebooks/Tests
current day is  2023-02-09
